In [1]:
!nvidia-smi

Thu Jul 29 06:56:00 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.42.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   41C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Directory 설정, 구글 드라이브 import

In [2]:
cur_dir = '/content/drive/MyDrive/KLUE_TC'

## Utils

In [3]:
!pip install adamp
!pip install transformers

  Created wheel for adamp: filename=adamp-0.3.0-py3-none-any.whl size=5998 sha256=a1b42e2e190a5a6b21eef44ddf7aa7d452fa5af8b1f70fd83e51e131d8a11e02
  Stored in directory: /root/.cache/pip/wheels/bb/95/21/ced2d2cb9944e3a72e58fece7958973eed3fd8d0aeb6e2e450
Successfully built adamp
     |████████████████████████████████| 2.6 MB 8.7 MB/s 
     |████████████████████████████████| 895 kB 49.6 MB/s 
     |████████████████████████████████| 3.3 MB 55.1 MB/s 
     |████████████████████████████████| 636 kB 81.6 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [4]:
import os
import random
import torch
import numpy as np
from torch import nn

from torch.optim import Adam, AdamW, SGD
from adamp import AdamP
from torch.optim.lr_scheduler import StepLR, ReduceLROnPlateau, CosineAnnealingLR, ExponentialLR, \
    CosineAnnealingWarmRestarts
from transformers import get_linear_schedule_with_warmup
from transformers import AutoConfig, AutoTokenizer, AutoModelForSequenceClassification
from sklearn import metrics


def set_seeds(seed=42):
    # 랜덤 시드를 설정하여 매 코드를 실행할 때마다 동일한 결과를 얻게 합니다.
    os.environ['PYTHONHASHSEED'] = str(seed)
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.cuda.manual_seed_all(seed)  # if use multi-GPU
    torch.backends.cudnn.benchmark = False


def load_tokenizer(args):
    tokenizer = AutoTokenizer.from_pretrained(
        args.tokenizer_name
        if args.tokenizer_name
        else args.model_name_or_path,
        use_fast=True,
    )

    return tokenizer


def load_model(args, model_name=None):
    if not model_name:
        model_name = args.model_name
    model_path = os.path.join(args.model_dir, model_name)
    print("Loading Model from:", model_path)
    load_state = torch.load(model_path)
    #load_state = torch.load(model_name)

    # Load pretrained model and tokenizer
    config = AutoConfig.from_pretrained(
        args.config_name
        if args.config_name
        else args.model_name_or_path,
    )

    config.num_labels = 7

    model = AutoModelForSequenceClassification.from_pretrained(
        args.model_name_or_path,
        from_tf=bool(".ckpt" in args.model_name_or_path),
        config=config,
    )

    model.classifier = nn.Sequential(
        nn.Linear(1024, 1024),
        nn.Dropout(p=0.3, inplace=False),
        nn.Linear(1024, 512),
        nn.Dropout(p=0.3, inplace=False),
        nn.Linear(512, 7),
    )

    model.load_state_dict(load_state['state_dict'], strict=True)

    model = model.to(args.device)

    print("Loading Model from:", model_path, "...Finished.")

    return model


def get_loaders(args, train, valid, is_inference=False):
    pin_memory = True
    train_loader, valid_loader = None, None

    if is_inference:
        test_dataset = YNAT_dataset(args, valid, is_inference)
        test_loader = torch.utils.data.DataLoader(test_dataset, num_workers=args.num_workers, shuffle=False,
                                                  batch_size=args.batch_size, pin_memory=pin_memory)
        return test_loader

    if train is not None:
        train_dataset = YNAT_dataset(args, train, is_inference)
        train_loader = torch.utils.data.DataLoader(train_dataset, num_workers=args.num_workers, shuffle=True,
                                                   batch_size=args.batch_size, pin_memory=pin_memory)
    if valid is not None:
        valid_dataset = YNAT_dataset(args, valid, is_inference)
        valid_loader = torch.utils.data.DataLoader(valid_dataset, num_workers=args.num_workers, shuffle=False,
                                                   batch_size=args.batch_size, pin_memory=pin_memory)

    return train_loader, valid_loader


# loss계산하고 parameter update!
def compute_loss(preds, targets, args):
    """
    Args :
        preds   : (batch_size, max_seq_len)
        targets : (batch_size, max_seq_len)
    """
    # print(preds, targets)
    loss = get_criterion(preds, targets, args)
    # 마지막 시퀀스에 대한 값만 loss 계산
    # loss = loss[:, -1]
    # loss = torch.mean(loss)
    return loss


def get_criterion(pred, target, args):
    if args.criterion == 'BCE':
        loss = nn.BCELoss(reduction="none")
    elif args.criterion == "BCELogit":
        loss = nn.BCEWithLogitsLoss(reduction="none")
    elif args.criterion == "MSE":
        loss = nn.MSELoss(reduction="none")
    elif args.criterion == "L1":
        loss = nn.L1Loss(reduction="none")
    elif args.criterion == "CE":
        loss = nn.CrossEntropyLoss()
    # NLL, CrossEntropy not available
    return loss(pred, target)


## Dataloader

In [5]:
import os
import torch
import pandas as pd


class Preprocess:
    def __init__(self, args):
        self.args = args
        self.train_data = None
        self.test_data = None

    def load_data(self, file_name):
        csv_file_name = os.path.join(self.args.data_dir, file_name)
        df = pd.read_csv(csv_file_name)
        #del df['Unnamed: 0']
        return df.values

    def load_train_data(self):
        self.train_data = self.load_data('train_data.csv')

    def load_test_data(self):
        self.test_data = self.load_data('test_data.csv')


class YNAT_dataset(torch.utils.data.Dataset):
    def __init__(self, args, data, is_inference):
        self.args = args
        self.data = data
        self.is_inference = is_inference

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        row = self.data[index]
        element = [row[i] for i in range(len(row))]
        #print(type(row))
        # np.array -> torch.tensor 형변환
        #for i, col in enumerate(row):
        #    if type(col) == str:
        #        pass
        #    else:
        #        row[i] = torch.tensor(col)

        return element

## Trainer

In [10]:
from sklearn.metrics import accuracy_score
from torch.nn.functional import one_hot
from tqdm import tqdm


def run(args, tokenizer, train_data, valid_data, cv_count):
    _, valid_loader = get_loaders(args, train_data, valid_data)

    if not args.cv_strategy:
        model_name = args.run_name
    else:
        model_name = f"{args.run_name.split('.pt')[0]}_{cv_count}.pt"

    tokenizer = load_tokenizer(args)

    model = load_model(args, model_name)
    model.eval()

    total_argmax_preds = []
    total_ids = []
    total_gts = []

    for step, batch in tqdm(enumerate(valid_loader), desc='Inferencing', total=len(valid_loader)):
        idx, text, label = batch

        tokenized_examples = tokenizer(
            text,
            max_length=args.max_seq_len,
            padding="max_length",
            return_tensors="pt"
        ).to(args.device)

        preds = model(**tokenized_examples)

        logits = preds['logits']
        logits = logits[:,0,:]
        argmax_logits = torch.argmax(logits, dim=1)

        if args.device == 'cuda':
            argmax_preds = argmax_logits.to('cpu').detach().numpy()
            preds = logits.to('cpu').detach().numpy()
        else:  # cpu
            argmax_preds = argmax_logits.detach().numpy()
            preds = logits.detach().numpy()

        total_argmax_preds += list(argmax_preds)
        total_ids += list(idx)
        total_gts += list(label)

    target_names = ['IT과학', '경제', '사회', '생활문화', '세계', '스포츠', '정치']
    print(metrics.classification_report(total_gts, total_argmax_preds, target_names=target_names))
    matrix = metrics.confusion_matrix(total_gts, total_argmax_preds)
    print(matrix)
    print(matrix.diagonal()/matrix.sum(axis=1))

## Train

In [11]:
import torch
from sklearn.model_selection import KFold, StratifiedKFold
from transformers import AutoConfig, AutoTokenizer, AutoModelForSequenceClassification
from datetime import datetime
from pytz import timezone


def main(args):
    if not args.run_name:
        args.run_name = datetime.now(timezone("Asia/Seoul")).strftime("%Y-%m-%d-%H:%M:%S")

    set_seeds(args.seed)

    device = "cuda" if torch.cuda.is_available() else "cpu"
    args.device = device

    tokenizer = AutoTokenizer.from_pretrained(
        args.tokenizer_name
        if args.tokenizer_name
        else args.model_name_or_path,
        use_fast=True,
    )

    preprocess = Preprocess(args)
    preprocess.load_train_data()
    train_data_origin = preprocess.train_data

    print(f"Size of train data : {len(train_data_origin)}")
    # print(f"size of test data : {len(test_data)}")

    if args.cv_strategy == 'random':
        kf = KFold(n_splits=args.fold_num, shuffle=True)
        splits = kf.split(X=train_data_origin)
    else:
        # default
        # 여기 각 label로 바꿔야됨
        train_labels = [sequence[-1] for sequence in train_data_origin]
        skf = StratifiedKFold(n_splits=args.fold_num, shuffle=True)
        splits = skf.split(X=train_data_origin, y=train_labels)

    acc_avg = 0
    for fold_num, (train_index, valid_index) in enumerate(splits):
        train_data = train_data_origin[train_index]
        valid_data = train_data_origin[valid_index]
        run(args, tokenizer, train_data, valid_data, fold_num + 1)

        if not args.cv_strategy:
            break

## Run

In [12]:
import argparse
import easydict

def parse_args():
    args = easydict.EasyDict({'run_name' : 'temp',
                             'seed':42,
                             'device' :'cuda',
                             'data_dir': cur_dir + '/data/open/',
                             'model_dir' : '/content/drive/MyDrive/KLUE_TC/models/',
                             'model_name_or_path' : 'klue/roberta-large',
                             'config_name' : None,
                             'tokenizer_name' : None,
                             'output_dir' : '/content/drive/MyDrive/KLUE_TC/output/',
                             
                             'accum_iter' : 4,
                             'gradient_accumulation' : True,

                             'cv_strategy' : 'stratified',
                             'fold_num' : 4,

                             'num_workers' : 1,

                             # 훈련
                             'n_epochs' : 3,
                             'batch_size' : 32,
                             'lr' : 1e-5,
                             'clip_grad' : 10,
                             'patience' : 5,
                             'max_seq_len' : 40,

                             # Optimizer
                             'optimizer' : 'adamW',

                             # Optimizer-parameters
                             'weight_decay' : 0.01,
                             'momentum' : 0.9,

                             # Scheduler
                             'scheduler' : 'step_lr',

                             # Scheduler-parameters
                             # plateau
                             'plateau_patience' : 10,
                             'plateau_factor' : 0.5,
                              
                             't_max' : 10,
                             'T_0' : 10,
                             'T_mult' : 2,
                             '--eta_min' : 0.01,

                             # linear_warmup
                             'warmup_ratio' : 0.3,

                             # Step LR
                             'step_size' : 50,
                             'gamma' : 0.1,

                             'criterion' : 'CE',

                             'log_steps' : 100})
    
    return args

In [13]:
if __name__ == '__main__':
    args = parse_args()
    main(args)

Size of train data : 45654
Loading Model from: /content/drive/MyDrive/KLUE_TC/models/temp_1.pt


Some weights of the model checkpoint at klue/roberta-large were not used when initializing RobertaForSequenceClassification: ['lm_head.dense.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.weight', 'lm_head.decoder.bias', 'lm_head.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at klue/roberta-large and are newly initialized: ['classifier.out_proj.bias', 'classifier.out_proj.weight', 'cl

Loading Model from: /content/drive/MyDrive/KLUE_TC/models/temp_1.pt ...Finished.


Inferencing: 100%|██████████| 357/357 [00:48<00:00,  7.41it/s]


              precision    recall  f1-score   support

        IT과학       0.83      0.88      0.85      1206
          경제       0.88      0.82      0.85      1555
          사회       0.81      0.79      0.80      1841
        생활문화       0.91      0.90      0.90      1483
          세계       0.90      0.95      0.93      1908
         스포츠       0.96      0.99      0.97      1734
          정치       0.94      0.90      0.92      1687

    accuracy                           0.89     11414
   macro avg       0.89      0.89      0.89     11414
weighted avg       0.89      0.89      0.89     11414

[[1058   44   51   14   31    5    3]
 [ 118 1275  106   16   28    7    5]
 [  68   98 1458   81   53   13   70]
 [  20   13   83 1332   24    7    4]
 [   8   18    6   15 1815   24   22]
 [   1    2    1    4    9 1716    1]
 [   3    4   91    1   46   18 1524]]
[0.87728027 0.81993569 0.79196089 0.89817937 0.95125786 0.98961938
 0.90337878]
Loading Model from: /content/drive/MyDrive/KLUE_TC/model

...

              precision    recall  f1-score   support

        IT과학       0.83      0.88      0.85      1206
          경제       0.87      0.86      0.87      1556
          사회       0.82      0.78      0.80      1840
        생활문화       0.88      0.92      0.90      1483
          세계       0.94      0.93      0.93      1907
         스포츠       0.96      0.98      0.97      1733
          정치       0.93      0.90      0.92      1688

    accuracy                           0.89     11413
   macro avg       0.89      0.89      0.89     11413
weighted avg       0.89      0.89      0.89     11413

[[1066   49   53   21   14    2    1]
 [  97 1343   75   17   19    1    4]
 [  77  112 1443  102   21   17   68]
 [  22    6   74 1366    9    2    4]
 [  12   22   20   27 1771   30   25]
 [  11    1    3    7    7 1695    9]
 [   4   13   93    5   41   11 1521]]
[0.88391376 0.86311054 0.78423913 0.92110587 0.9286838  0.97807271
 0.90106635]
Loading Model from: /content/drive/MyDrive/KLUE_TC/model

Some weights of the model checkpoint at klue/roberta-large were not used when initializing RobertaForSequenceClassification: ['lm_head.dense.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.weight', 'lm_head.decoder.bias', 'lm_head.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at klue/roberta-large and are newly initialized: ['classifier.out_proj.bias', 'classifier.out_proj.weight', 'cl

Loading Model from: /content/drive/MyDrive/KLUE_TC/models/temp_4.pt ...Finished.


Inferencing: 100%|██████████| 357/357 [00:48<00:00,  7.39it/s]


              precision    recall  f1-score   support

        IT과학       0.85      0.85      0.85      1206
          경제       0.84      0.86      0.85      1555
          사회       0.83      0.77      0.80      1840
        생활문화       0.91      0.90      0.90      1484
          세계       0.92      0.94      0.93      1907
         스포츠       0.96      0.98      0.97      1733
          정치       0.92      0.93      0.92      1688

    accuracy                           0.89     11413
   macro avg       0.89      0.89      0.89     11413
weighted avg       0.89      0.89      0.89     11413

[[1030   84   57   13   10    9    3]
 [  88 1338   76   11   27    4   11]
 [  60  128 1419   85   41   15   92]
 [  20   17   88 1332   16    5    6]
 [  13   26    8   14 1793   27   26]
 [   0    2   15    7    3 1702    4]
 [   3    6   56    1   50   10 1562]]
[0.85406302 0.86045016 0.77119565 0.89757412 0.94022024 0.98211194
 0.92535545]


In [14]:
!pip freeze

absl-py==0.12.0
adamp==0.3.0
alabaster==0.7.12
albumentations==0.1.12
altair==4.1.0
appdirs==1.4.4
argon2-cffi==20.1.0
arviz==0.11.2
astor==0.8.1
astropy==4.2.1
astunparse==1.6.3
async-generator==1.10
atari-py==0.2.9
atomicwrites==1.4.0
attrs==21.2.0
audioread==2.1.9
autograd==1.3
Babel==2.9.1
backcall==0.2.0
beautifulsoup4==4.6.3
bleach==3.3.0
blis==0.4.1
bokeh==2.3.3
Bottleneck==1.3.2
branca==0.4.2
bs4==0.0.1
CacheControl==0.12.6
cached-property==1.5.2
cachetools==4.2.2
catalogue==1.0.0
certifi==2021.5.30
cffi==1.14.6
cftime==1.5.0
chardet==3.0.4
charset-normalizer==2.0.2
click==7.1.2
cloudpickle==1.3.0
cmake==3.12.0
cmdstanpy==0.9.5
colorcet==2.0.6
colorlover==0.3.0
community==1.0.0b1
contextlib2==0.5.5
convertdate==2.3.2
coverage==3.7.1
coveralls==0.5
crcmod==1.7
cufflinks==0.17.3
cupy-cuda101==9.1.0
cvxopt==1.2.6
cvxpy==1.0.31
cycler==0.10.0
cymem==2.0.5
Cython==0.29.23
daft==0.0.4
dask==2.12.0
datascience==0.10.6
debugpy==1.0.0
decorator==4.4.2
defusedxml==0.7.1
descartes==1.1.0
